# 📊 Day 1-00.02: 데이터 탐색하기 (초보자용)

## 🎯 이번 노트북에서 할 일
- **한국어 데이터셋**을 Hugging Face에서 가져오기
- **데이터가 어떤 모양**인지 살펴보기
- **질문과 답변**이 어떻게 구성되어 있는지 확인하기
- **데이터 시각화**로 이해하기 쉽게 만들기

## 💡 오늘 사용할 데이터
- **데이터셋**: `neural-bridge/rag-dataset-12000`
- **언어**: 한국어
- **용도**: RAG(검색 증강 생성) 파인튜닝
- **구조**: 질문 + 답변 + 관련 문서


## 1. 필요한 라이브러리 불러오기


In [ ]:
# 데이터 탐색에 필요한 라이브러리들을 불러옵니다
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib) - 그래프에서 한글이 깨지지 않도록 설정
# Colab 환경에서 나눔 글꼴을 설치하고 matplotlib에 적용
print("🔧 한글 폰트 설정 중...")
!apt-get update -qq
!apt-get install fonts-nanum -qq > /dev/null

import matplotlib.font_manager as fm

# 나눔바른고딕 폰트 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# 폰트 매니저에 폰트 추가 - 그래프에서 한글 표시를 위해 필요
fm.fontManager.addfont(fontpath)

# matplotlib 설정 업데이트 - 모든 그래프에서 한글이 정상적으로 표시됨
plt.rcParams.update({
    'font.family': 'NanumBarunGothic',  # 기본 폰트를 나눔바른고딕으로 설정
    'axes.unicode_minus': False         # 음수 기호 표시 문제 해결
})

print("✅ 한글 폰트 설정 완료 - 그래프에서 한글이 정상 표시됩니다")
print("📦 라이브러리 import 완료!")


## 2. Hugging Face에서 데이터셋 가져오기


In [ ]:
# Hugging Face에서 한국어 RAG 데이터셋을 가져옵니다
print("📥 데이터셋을 다운로드하는 중...")

# 데이터셋 로드 (처음에는 작은 샘플만 가져와서 확인)
dataset = load_dataset("neural-bridge/rag-dataset-12000", split="train")

print(f"✅ 데이터셋 로드 완료!")
print(f"📊 총 데이터 개수: {len(dataset):,}개")
print(f"🔍 데이터 타입: {type(dataset)}")
print(f"📋 데이터 구조: {dataset.features}")


## 3. 데이터 구조 살펴보기


In [ ]:
# 첫 번째 데이터 샘플을 자세히 살펴봅시다
print("🔍 첫 번째 데이터 샘플:")
print("=" * 50)

# 첫 번째 샘플 가져오기
sample = dataset[0]

# 각 필드별로 출력
for key, value in sample.items():
    print(f"\n📌 {key}:")
    print(f"   타입: {type(value)}")
    if isinstance(value, str):
        # 문자열인 경우 길이와 일부 내용 표시
        print(f"   길이: {len(value)} 글자")
        print(f"   내용 미리보기: {value[:100]}...")
    elif isinstance(value, list):
        # 리스트인 경우 길이와 첫 번째 항목 표시
        print(f"   개수: {len(value)}개")
        if value:
            print(f"   첫 번째 항목: {value[0][:100]}...")
    else:
        print(f"   값: {value}")
    print("-" * 30)


## 4. 여러 샘플 살펴보기


In [ ]:
# 처음 3개의 샘플을 예쁘게 표시해봅시다
print("📚 처음 3개 샘플:")
print("=" * 80)

for i in range(min(3, len(dataset))):
    sample = dataset[i]
    print(f"\n🔹 샘플 {i+1}:")
    print(f"   질문: {sample['question']}")
    print(f"   답변: {sample['answer'][:150]}...")
    print(f"   관련 문서 개수: {len(sample['context'])}")
    if sample['context']:
        print(f"   첫 번째 문서: {sample['context'][0][:100]}...")
    print("-" * 80)


## 5. 데이터 통계 분석하기


In [ ]:
# 데이터의 기본 통계를 계산해봅시다
print("📊 데이터 통계 분석:")
print("=" * 50)

# 먼저 None 값이 있는지 확인해보겠습니다
print("🔍 데이터 품질 확인:")
none_questions = sum(1 for sample in dataset if sample['question'] is None)
none_answers = sum(1 for sample in dataset if sample['answer'] is None)
none_contexts = sum(1 for sample in dataset if sample['context'] is None)

print(f"   None 질문: {none_questions}개")
print(f"   None 답변: {none_answers}개") 
print(f"   None 컨텍스트: {none_contexts}개")

# None이 아닌 값들만으로 통계 계산
question_lengths = [len(sample['question']) for sample in dataset if sample['question'] is not None]
answer_lengths = [len(sample['answer']) for sample in dataset if sample['answer'] is not None]
context_counts = [len(sample['context']) for sample in dataset if sample['context'] is not None]

print(f"\n📝 질문 길이:")
print(f"   평균: {np.mean(question_lengths):.1f} 글자")
print(f"   최소: {np.min(question_lengths)} 글자")
print(f"   최대: {np.max(question_lengths)} 글자")

print(f"\n💬 답변 길이:")
print(f"   평균: {np.mean(answer_lengths):.1f} 글자")
print(f"   최소: {np.min(answer_lengths)} 글자")
print(f"   최대: {np.max(answer_lengths)} 글자")

print(f"\n📄 관련 문서 개수:")
print(f"   평균: {np.mean(context_counts):.1f}개")
print(f"   최소: {np.min(context_counts)}개")
print(f"   최대: {np.max(context_counts)}개")


## 6. 데이터 시각화하기


In [ ]:
# 데이터 분포를 그래프로 그려봅시다
# 한글 폰트 설정 확인 및 적용
plt.rcParams.update({
    'font.family': 'NanumBarunGothic',
    'axes.unicode_minus': False
})

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📊 RAG 데이터셋 분석', fontsize=16, fontweight='bold')

# 1. 질문 길이 분포
axes[0, 0].hist(question_lengths, bins=30, alpha=0.7, color='skyblue')
axes[0, 0].set_title('질문 길이 분포')
axes[0, 0].set_xlabel('글자 수')
axes[0, 0].set_ylabel('개수')

# 2. 답변 길이 분포
axes[0, 1].hist(answer_lengths, bins=30, alpha=0.7, color='lightgreen')
axes[0, 1].set_title('답변 길이 분포')
axes[0, 1].set_xlabel('글자 수')
axes[0, 1].set_ylabel('개수')

# 3. 관련 문서 개수 분포
axes[1, 0].hist(context_counts, bins=20, alpha=0.7, color='orange')
axes[1, 0].set_title('관련 문서 개수 분포')
axes[1, 0].set_xlabel('문서 개수')
axes[1, 0].set_ylabel('개수')

# 4. 질문 vs 답변 길이 관계
axes[1, 1].scatter(question_lengths, answer_lengths, alpha=0.5, color='purple')
axes[1, 1].set_title('질문 길이 vs 답변 길이')
axes[1, 1].set_xlabel('질문 길이 (글자)')
axes[1, 1].set_ylabel('답변 길이 (글자)')

plt.tight_layout()
plt.show()

print("✅ 데이터 시각화 완료!")


## 7. 데이터 저장하기


In [ ]:
# 탐색한 데이터를 저장해둡시다 (다음 단계에서 사용할 예정)
print("💾 데이터를 저장하는 중...")

# 전체 데이터셋을 저장
dataset.save_to_disk("data/rag_dataset")

# 통계 정보도 저장
stats = {
    "total_samples": len(dataset),
    "avg_question_length": np.mean(question_lengths),
    "avg_answer_length": np.mean(answer_lengths),
    "avg_context_count": np.mean(context_counts),
    "question_lengths": question_lengths,
    "answer_lengths": answer_lengths,
    "context_counts": context_counts
}

with open("data/dataset_stats.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print("✅ 데이터 저장 완료!")
print("   - data/rag_dataset/: 전체 데이터셋")
print("   - data/dataset_stats.json: 통계 정보")


## 8. 다음 단계 안내

### 🎯 다음 노트북에서 할 일
**00.03-raft-preprocessing.ipynb**에서:
1. **RAFT 템플릿** 만들기
2. **데이터를 파인튜닝용으로 변환**하기
3. **토큰화** 준비하기

### 💡 지금까지 배운 것
- ✅ 한국어 RAG 데이터셋의 구조 이해
- ✅ 질문-답변-문서의 관계 파악
- ✅ 데이터 분포와 통계 분석
- ✅ 시각화를 통한 데이터 이해

### 🚀 준비 완료!
이제 다음 노트북으로 넘어가서 데이터를 파인튜닝용으로 변환해보겠습니다!
